In [1]:
import pandas as pd
import numpy as np
import regex as re
import warnings
warnings.filterwarnings('ignore')

In [2]:
a = pd.read_csv('../data/actor.csv', encoding='latin1')
actor=a.copy()
c = pd.read_csv('../data/category.csv', encoding='latin1')
category=c.copy()
f = pd.read_csv('../data/film.csv', encoding='latin1')
film=f.copy()
inventory = pd.read_csv('../data/inventory.csv', encoding='latin1')
i=inventory.copy()
l = pd.read_csv('../data/language.csv', encoding='latin1')
language=l.copy()
old_HDD = pd.read_csv('../data/old_HDD.csv', encoding='latin1')
o=old_HDD.copy()
rental = pd.read_csv('../data/rental.csv', encoding='latin1')
r=rental.copy()

## Exploración archivos y limpieza previa.

In [3]:
#1º. Actores.

a.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   actor_id     200 non-null    int64 
 1   first_name   200 non-null    object
 2   last_name    200 non-null    object
 3   last_update  200 non-null    object
dtypes: int64(1), object(3)
memory usage: 41.0 KB


In [4]:
nulos=a.isnull().sum() #No hay ningun valor nulo
nulos

actor_id       0
first_name     0
last_name      0
last_update    0
dtype: int64

In [5]:
a.columns #vamos a ponerlo bonito

Index(['actor_id', 'first_name', 'last_name', 'last_update'], dtype='object')

In [6]:
a.columns=a.columns.str.replace('_',' ').str.title()
a.columns

Index(['Actor Id', 'First Name', 'Last Name', 'Last Update'], dtype='object')

In [7]:
a['Last Update'].head() #toda la columna es igual y no nos da ninguna información relevante, nos la cargamos 🆑

0    2006-02-15 04:34:33
1    2006-02-15 04:34:33
2    2006-02-15 04:34:33
3    2006-02-15 04:34:33
4    2006-02-15 04:34:33
Name: Last Update, dtype: object

In [8]:
a.columns.duplicated().any() #sin duplicados

False

In [9]:
a.drop(columns=['Last Update'], inplace=True)
a.columns

Index(['Actor Id', 'First Name', 'Last Name'], dtype='object')

In [10]:
a[a.duplicated(subset=['First Name', 'Last Name'],keep=False)] # busco duplicados pero me centro solo en los nombes ya que los id son diferentes para cada uno,
                                                               # hay un actor repetido, vamos a quitarlo. El keep False lo saque del chat 🤫, porque si no
                                                               #lo ponía solo me mostraba el 109 con el nombre y no sabia de donde venía el duplicado.

,Actor Id,First Name,Last Name
100,101,SUSAN,DAVIS
109,110,SUSAN,DAVIS


In [11]:
a.drop([109],axis=0,inplace=True) # no me va a interferir porque no hay ninguna tabla que me relacione su id con nada, no peta 💣

In [22]:
a.reset_index(inplace=True,drop=True)
nuevos_id = pd.Series(range(1, len(a)+1)) # cambio los id para que no tener salto y reseteo index
a['Actor Id']=nuevos_id

a.iloc[107:112] # compruebo que no me pegue el salto, he hecho todo esto porque para el análisis me puede servir

,Actor Id,First Name,Last Name
107,108,WARREN,NOLTE
108,109,SYLVESTER,DERN
109,110,CAMERON,ZELLWEGER
110,111,RUSSELL,BACALL
111,112,MORGAN,HOPKINS


In [ ]:
#2º) Categoría: 

